# HHH eval with and without CoT

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-hf")

In [ ]:
from datasets import load_dataset

hhh_dataset = load_dataset("HuggingFaceH4/hhh_alignment")

In [ ]:
YAML_boolq_string = '''
task: demo_boolq
dataset_path: super_glue
dataset_name: boolq
output_type: multiple_choice
training_split: train
validation_split: validation
doc_to_text: "{{passage}}\nQuestion: {{question}}?\nAnswer:"
doc_to_target: label
doc_to_choice: ["no", "yes"]
should_decontaminate: true
doc_to_decontamination_query: passage
metric_list:
  - metric: acc
'''
with open('boolq.yaml', 'w') as f:
    f.write(YAML_boolq_string)

In [ ]:
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_boolq \
    --limit 10